# 第7章 图像数据处理
在第6章中详细介绍了CNN，并提到CNN给图像识别技术带来了突破性进展。这一章将从另外一个维度来进一步提升图像识别的精度以及训练的速度。

喜欢摄影的读者都知道图像的亮度、对比度等属性对图像的影响是非常大的，相同物体在不同亮度、对比度下差别非常大。然而在很多图像识别问题中，这些因素都不应该影响最后的识别结果。所以**本章将介绍如何对图像数据进行预处理使训练得到的神经网络模型尽可能小地被无关因素所影响。**

但与此同时，复杂的预处理过程可能导致训练效率的下降。为了减小预处理对于训练速度的影响，在本章中也将详细地介绍TensorFlow 中**多线程处理输入数据**的解决方案。

## 7.1 TFRecord输入数据格式
6.5节中给出了一个程序来处理花朵分类的数据。在这个程序中，使用了一个从类别名称到所有数据列表的词典来维护图像和类别的关系。这种方式的**可扩展性非常差**，当数据来源更加复杂、每一个样例中的信息更加丰富之后，这种方式就很难有效地记录输入数据中的信息了。于是TensorFlow提供了一种统一的格式来存储数据——**TFRecord**。

### 7.1.1 TFRecord格式介绍
TFRecord文件中的数据都是通过tf.train.Example Protocol Buffer的格式存储的。以下给出了tf.train.Example的定义：

In [ ]:
'''
message Example {
    Feature features = 1;
};

message Features {
    map<string, Feature> feature = 1;
};

message Feature {
    oneof kind {
        BytesList bytes_list = 1;
        FloatList float_list = 2;
        Int64List int64_list = 3;
  }
};
'''

可以看出tf.train.Example的数据结构是比较简洁的。**tf.train.Example中包含了一个从属性名称到取值的字典**。其中属性名称为一个字符串，属性的取值可以为字符串（BytesList）、实数列表（FloatList）或者整数列表（Int64List）。比如将一张解码前的图像存为一个字符串，图像所对应的类别编号存为整数列表。

** 7.1.2 TFRecord样例程序**
以下程序给出了以MNIST数据集为例，如何将其转化为TFRecord格式，及读取这个TFRecord格式。

**1. 写入TFRecord格式**：

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

# 生成整数型的属性
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# 生成字符串型的属性
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# 将数据转化为tf.train.Example格式。
def _make_example(pixels, label, image):
    image_raw = image.tostring()
    example = tf.train.Example(features=tf.train.Features(feature={
        'pixels': _int64_feature(pixels),
        'label': _int64_feature(np.argmax(label)),
        'image_raw': _bytes_feature(image_raw)
    }))
    return example

# 读取mnist训练数据。
mnist = input_data.read_data_sets("../../datasets/MNIST_data", 
                                  dtype=tf.uint8, 
                                  one_hot=True)
images = mnist.train.images
labels = mnist.train.labels
pixels = images.shape[1]
num_examples = mnist.train.num_examples

# 输出包含训练数据的TFRecord文件。
# 先创建一个writer来写TFRecord文件
with tf.python_io.TFRecordWriter(path="output.tfrecords") as writer:
    for index in range(num_examples):
        # 将一个样例转化为Example Protocol Buffer，并将所有的信息写入这个数据结构
        example = _make_example(pixels, labels[index], images[index])
        # 将一个Example写入TFRecord文件
        writer.write(example.SerializeToString())
print("TFRecord训练文件已保存。")

# 读取mnist测试数据。
images_test = mnist.test.images
labels_test = mnist.test.labels
pixels_test = images_test.shape[1]
num_examples_test = mnist.test.num_examples

# 输出包含测试数据的TFRecord文件。
with tf.python_io.TFRecordWriter(path="output_test.tfrecords") as writer:
    for index in range(num_examples_test):
        example = _make_example(
            pixels_test, labels_test[index], images_test[index])
        writer.write(example.SerializeToString())
print("TFRecord测试文件已保存。")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../datasets/MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../datasets/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../datasets/MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
TFRecord训练文件已保存。
TFRecord测试文件已保存。


当数据量较大时，也可以将数据写入多个TFRecord文件。TensorFlow对文件列表读取数据提供了很好的支持，这在7.3.2节中会介绍。

**2. 读取TFRecord文件**:

In [1]:
import tensorflow as tf

# 创建一个reader来读取TFRecord文件中的样例
reader = tf.TFRecordReader()
# 创建一个列表来维护输入文件列表，在7.3.2节中会详细介绍
filename_queue = tf.train.string_input_producer(["output.tfrecords"])

# 从文件中读取一个样例。也可以使用read_up_to函数一次性读取多个样例
_, serialized_example = reader.read(filename_queue)

# 解析读取的样例。如果需要解析多个样例，可使用parse_example函数
features = tf.parse_single_example(
    serialized_example,
    features={
        # TensorFlow提供两种不同的属性解析方法。
        # 一种是方法是tf.FixedLenFeature,这种方法解析的结果为一个Tensor。
        # 另一种方法是tf.VarLenFeature，这种方法得到的解析结果为SparseTensor，用于处理稀疏数据。
        # 这里解析数据的格式需要和上面程序写入数据的格式一致。
        'image_raw':tf.FixedLenFeature([],tf.string),
        'pixels':tf.FixedLenFeature([],tf.int64),
        'label':tf.FixedLenFeature([],tf.int64)
    })

# tf.decode_raw函数可以将字符串解析成图像对应的像素数组
images = tf.decode_raw(features['image_raw'],tf.uint8)
labels = tf.cast(features['label'],tf.int32)
pixels = tf.cast(features['pixels'],tf.int32)

sess = tf.Session()
# 启动多线程处理输入数据。7.3节将会更详细介绍多线程
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

# 每次运行可以读取TFRecord文件中的一个样例。当所有样例都读完之后，在此样例中程序会再重头读取
for i in range(10):
    image, label, pixel = sess.run([images, labels, pixels])
print(label, pixel, image)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
8 784 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  22  69
 148 210 253 156 122   7   0   0  18   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0 100 221 252 252 253 252 252 252 113   0   0
 185   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  31 221
 252 252 244 23